In [ ]:
import pyodbc
import pandas as pd
import numpy as np
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=mecshsql.database.windows.net,1433;DATABASE=datamart;UID=Channel;PWD=Passw0rd')
sql = """

SELECT
	 A.[Product]
	,A.[Campaign]
	,A.[StartDate]
	,A.[StdMedia]
	,A.[Std.Format]
	,A.[Target_Market]
	,A.Net_Media_Cost As Media_Cost
FROM [dbo].[MediaPlan_CampaignPlacements_Chanel] A
where 
"""
df_plac_perf = pd.read_sql(sql,cnxn)


Problem optimize marketing budget for conversion rates of your product/service by 
using pay per click providers like adwords search, adwords display network,
facebooks ads and linkedin ads, # we model the problem as 
linear mathematical program in python  and pulp solver was for solving linear program

objective fucntion: maximize conversion rates
desicion variables: amount to spend in each provider
constraints: 
maximum budget per ppc provider is 500 euros
maximum Total budget is 1800 euros
program dependency is pulp solver https://code.google.com/p/pulp-or/


In [83]:
from pulp import *

# initialize the model maximize conversion rate 
marketing_budget = pulp.LpProblem('Pay Per Click Budget Optimization', pulp.LpMaximize)

# list of ppc providers
ppc_providers = ['Adwords_Search', 'Adwords_Display', 'Facebook_Ads', 'Linkedin_Ads']

# the decision variables that will be spend in every ppc provider 
x = pulp.LpVariable.dict('x_%s', ppc_providers, lowBound =0)


total_budget= 10000.0
# average cost per click by ppc provider 0.55 eurs etc
avg_cpm = dict(zip(ppc_providers, [0.55, 0.35, 0.75, 0.60]))

# average conversion rates by ppc provider
avg_conv_rate = dict(zip(ppc_providers, [0.11, 0.20, 0.08, 0.14]))

# objective function max conversion rates
marketing_budget += sum([ (x[i]*total_budget/avg_cpm[i])*avg_conv_rate[i] for i in ppc_providers])

# total budget constraint < 1800 euros
marketing_budget += sum([ x[i] for i in ppc_providers]) <= 1

# adwords search contstraing  equal or less than 500 euros
marketing_budget += x['Adwords_Search'] >=.1

marketing_budget += x['Adwords_Display'] <= .8

marketing_budget += x['Facebook_Ads'] <= .5

marketing_budget += x['Linkedin_Ads'] <= .8


#we using the default solver
marketing_budget.solve()

#print the result
for ppc_provider in ppc_providers:
	print('The amount of euros for %s is %s '%(ppc_provider,x[ppc_provider].value()))

The amount of euros for Adwords_Search is 0.1 
The amount of euros for Adwords_Display is 0.8 
The amount of euros for Facebook_Ads is 0.0 
The amount of euros for Linkedin_Ads is 0.1 


In [78]:
import pandas as pd
df_cpm = pd.DataFrame.from_dict(avg_cpm, orient='index').rename(columns={0:'cpm'})

df_ctr = pd.DataFrame.from_dict(avg_conv_rate, orient='index').rename(columns={0:'ctr'})
df_rpt= pd.concat([df_cpm,df_ctr], axis=1)

df_rpt['cpc'] = df_rpt['cpm'] * df_rpt['ctr']
df_rpt

,cpm,ctr,cpc
Adwords_Search,0.55,0.11,0.0605
Adwords_Display,0.35,0.20,0.0700
Facebook_Ads,0.75,0.08,0.0600
Linkedin_Ads,0.60,0.14,0.0840
